In [ ]:
!pip install --upgrade unsloth torch torchvision

In [ ]:
import torch
from unsloth import FastLanguageModel
from torch.utils.data import Dataset, DataLoader
import numpy as np

device = torch.device('cuda')

In [3]:
class CustomerDataset(Dataset):
    def __init__(self, users, sessions, products):
        self.users = users
        self.sessions = sessions
        self.products = products

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return {
            'user_embedding': torch.tensor(self.users[idx], dtype=torch.float32),
            'session_embedding': torch.tensor(self.sessions[idx], dtype=torch.float32),
            'product_embeddings': torch.tensor(self.products[idx], dtype=torch.float32)
        }

In [4]:
users = np.random.randn(100, 128)
sessions = np.random.randn(100, 64)
products = np.random.randn(100, 50, 128)

dataset = CustomerDataset(users, sessions, products)
loader = DataLoader(dataset, batch_size=4, shuffle=True)

In [ ]:
model_name = "EleutherAI/gpt-neo-125M"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    dtype=torch.float16,
    max_seq_length=1024,
    full_finetuning=False,
    attn_implementation="eager"
)


In [ ]:
import torch.nn as nn

proj = nn.Linear(state.shape[1], prod.shape[2]).to(device)

for batch in loader:
    user = batch['user_embedding'].to(device)
    session = batch['session_embedding'].to(device)
    prod = batch['product_embeddings'].to(device)

    state = torch.cat([user, session], dim=1)
    state = proj(state)

    state_norm = F.normalize(state, dim=1).unsqueeze(1)
    prod_norm  = F.normalize(prod, dim=2)
    sim = torch.bmm(state_norm, prod_norm.transpose(1, 2)).squeeze(1)

    recommended_idx = torch.argmax(sim, dim=1)
    recommended_products = prod[range(prod.shape[0]), recommended_idx]


In [ ]:
all_recommendations = []

for batch in loader:
    user = batch['user_embedding'].to(device)
    session = batch['session_embedding'].to(device)
    prod = batch['product_embeddings'].to(device)

    state = torch.cat([user, session], dim=1)
    state = proj(state)

    state_norm = F.normalize(state, dim=1).unsqueeze(1)
    prod_norm  = F.normalize(prod, dim=2)
    sim = torch.bmm(state_norm, prod_norm.transpose(1, 2)).squeeze(1)

    recommended_idx = torch.argmax(sim, dim=1)
    recommended_products = prod[range(prod.shape[0]), recommended_idx]

    for i, idx in enumerate(recommended_idx):
      all_recommendations.append({
          "user_idx_in_batch": i,
          "recommended_product_idx": idx.item(),
          "similarity_score": sim[i, idx].item()
      })

print("Done, son.\n")
print("Sample recommendations:")
for r in all_recommendations[:5]:
    print(r)


In [ ]:
import torch
import torch.nn.functional as F
import random

device = "cuda" if torch.cuda.is_available() else "cpu"

num_products = 1000
embedding_dim = 128
product_names = [f"{style}_{i}" for i, style in enumerate(
    ["backpack", "handbag", "tote", "crossbody", "duffel", "messenger", "clutch", "satchel", "hobo", "bucket"] * 100
)]
product_embeddings = torch.rand(num_products, embedding_dim, device=device)

batch_size = 4
num_batches = 3
proj = torch.nn.Linear(embedding_dim * 2, embedding_dim).to(device)

num_candidates = 5
candidate_indices = random.sample(range(num_products), num_candidates)

all_recommendations = {}

for candidate_idx in candidate_indices:
    candidate_name = product_names[candidate_idx]
    candidate_embedding = product_embeddings[candidate_idx].unsqueeze(0).repeat(batch_size, 1)

    recommendations_per_candidate = []

    for batch_idx in range(num_batches):
        user = torch.rand(batch_size, embedding_dim, device=device)
        session = torch.rand(batch_size, embedding_dim, device=device)

        state = torch.cat([user, session], dim=1)
        state = proj(state)

        combined_state = state + candidate_embedding

        state_norm = F.normalize(combined_state, dim=1).unsqueeze(1)
        prod_norm_batch = F.normalize(product_embeddings.unsqueeze(0), dim=2).expand(batch_size, -1, -1)
        sim = torch.bmm(state_norm, prod_norm_batch.transpose(1, 2)).squeeze(1)

        topk = torch.topk(sim, k=3, dim=1)
        for i, indices in enumerate(topk.indices):
            recommendations_per_candidate.append({
                "user_idx_in_batch": i + batch_idx * batch_size,
                "candidate_product": candidate_name,
                "top_recommended_products": [product_names[idx] for idx in indices],
                "similarity_scores": [sim[i, idx].item() for idx in indices]
            })

    all_recommendations[candidate_name] = recommendations_per_candidate

for candidate, recs in list(all_recommendations.items())[:3]:
    print(f"\nRecommendationz: {candidate}")
    for r in recs[:3]:
        print(r)
